In [ ]:
# Install packages from requirements.txt
%pip install -r requirements.txt

In [40]:
# import modules

import os
import json
import shutil
import requests
import glob
import openpyxl

import numpy as np
import pandas as pd

from pymongo import MongoClient
from kaggle.api.kaggle_api_extended import KaggleApi

In [3]:
# 1 - CSV (.csv)

# Define URL and target file path
url = "https://corgis-edu.github.io/corgis/datasets/csv/hospitals/hospitals.csv"
save_dir = "./data/csv"
file_name = "hospitals.csv"
file_path = os.path.join(save_dir, file_name)

# Create directory if it doesn't exist
os.makedirs(save_dir, exist_ok=True)

# Check if file exists, delete if it does
if os.path.exists(file_path):
    print(f"File {file_path} already exists. Deleting it...")
    os.remove(file_path)

# Download the file
print(f"Downloading from {url}...")
response = requests.get(url)
response.raise_for_status()  # Raise an error for bad status

# Save the file
with open(file_path, 'wb') as f:
    f.write(response.content)

print(f"File saved to {file_path}")

File saved to ./data/csv\hospitals.csv


In [4]:
# 2 - Excel (.xlsx)

# Direct file URL extracted from the Google redirect link
url = "https://www.cms.gov/files/document/2021-reporting-cycle-teaching-hospital-listx.xlsx"
save_dir = "./data/excel"
file_name = "hospitals.xlsx"
file_path = os.path.join(save_dir, file_name)

# Create the directory if it doesn't exist
os.makedirs(save_dir, exist_ok=True)

# If file exists, delete it
if os.path.exists(file_path):
    print(f"File already exists at {file_path}. Deleting...")
    os.remove(file_path)

# Download the file
print(f"Downloading from {url}...")
response = requests.get(url)
response.raise_for_status()

# Save the file
with open(file_path, 'wb') as f:
    f.write(response.content)

print(f"File downloaded and saved to {file_path}")

File downloaded and saved to ./data/excel\hospitals.xlsx


In [5]:
# 3 - JSON (.json)

# URL of the JSON file
url = "https://raw.githubusercontent.com/dariusk/corpora/master/data/medicine/hospitals.json"
save_dir = "./data/json"
file_name = "hospitals.json"
file_path = os.path.join(save_dir, file_name)

# Create directory if it doesn't exist
os.makedirs(save_dir, exist_ok=True)

# Delete existing file if it exists
if os.path.exists(file_path):
    print(f"File already exists at {file_path}. Deleting...")
    os.remove(file_path)

# Download the file
print(f"Downloading from {url}...")
response = requests.get(url)
response.raise_for_status()

# Save the file
with open(file_path, 'wb') as f:
    f.write(response.content)

print(f"File downloaded and saved to {file_path}")


File downloaded and saved to ./data/json\hospitals.json


In [7]:
# 4 - Database (SQL/NOSQL) *NoSQL [MongoDB]

!python script.py

Created directory: ./data/mongo
Download complete: ./data/mongo/hospitals.csv
Dropped existing collection: hospital
Reading CSV: ./data/mongo/hospitals.csv
Inserting 6064 records into MongoDB...
Data inserted successfully.


In [ ]:
# 5 - API/Kaggle 
 
# ✅ Update this to where kaggle.json is actually located
kaggle_json_src = "./config/kaggle.json"

# Target directory and destination path
kaggle_config_dir = "C:/Users/User/.kaggle"  # Recommended location for kaggle.json
kaggle_json_dest = os.path.join(kaggle_config_dir, "kaggle.json")

# Create directory if it doesn't exist
os.makedirs(kaggle_config_dir, exist_ok=True)

# Move the file if it exists
if os.path.exists(kaggle_json_src):
    shutil.move(kaggle_json_src, kaggle_json_dest)
    print("✅ kaggle.json moved successfully.")
else:
    raise FileNotFoundError(f"🚫 File not found: {kaggle_json_src}")

# Set environment variable
os.environ['KAGGLE_CONFIG_DIR'] = kaggle_config_dir

# Set permissions (may not be strictly necessary on Windows)
try:
    os.chmod(kaggle_json_dest, 0o600)
except PermissionError:
    print("⚠️ Could not change permissions on Windows (safe to ignore)")

# Now you can proceed to authenticate with Kaggle
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()
print("✅ Kaggle API authenticated successfully.")

✅ kaggle.json moved successfully.
✅ Kaggle API authenticated successfully.


In [22]:
# Set the path where kaggle.json is located (adjust if needed)
# If in Colab, upload kaggle.json first and specify its location
kaggle_json_path = "C:/Users/User/.kaggle"  # Change this if your file is elsewhere

# Ensure the Kaggle config directory exists
kaggle_config_dir = "C:/Users/User/.kaggle" # Default Kaggle config dir in Colab
os.makedirs(kaggle_config_dir, exist_ok=True)

# Move kaggle.json to the expected location if it's not there
if os.path.exists(kaggle_json_path):
    os.system(f"mv {kaggle_json_path} {kaggle_config_dir}/kaggle.json")
    os.system(f"chmod 600 {kaggle_config_dir}/kaggle.json")  # Set correct permissions
else:
    print("Please upload kaggle.json to /content/ and rerun this cell")
    raise FileNotFoundError("kaggle.json not found at /content/")

# Set environment variable for Kaggle API
os.environ['KAGGLE_CONFIG_DIR'] = kaggle_config_dir

# Initialize and authenticate API
api = KaggleApi()
try:
    api.authenticate()
    print("Kaggle authentication successful")
    auth_valid = True
except Exception as e:
    print(f"Authentication failed: {str(e)}")
    auth_valid = False

# ✅ Download dataset after successful authentication
if auth_valid:
    dataset_name = "thedevastator/hospitals-in-the-united-states-a-comprehensive-d"  # 👉 Replace with your desired dataset name
    download_path = "./data/kaggle"
    os.makedirs(download_path, exist_ok=True)

    print(f"Downloading Kaggle dataset '{dataset_name}' to '{download_path}'...")
    api.dataset_download_files(dataset_name, path=download_path, unzip=True)
    print("✅ Dataset download and extraction complete.")

Kaggle authentication successful
Dataset URL: https://www.kaggle.com/datasets/thedevastator/hospitals-in-the-united-states-a-comprehensive-d
✅ Dataset download and extraction complete.


In [23]:
# Transforming Data

# Target columns
TARGET_COLUMNS = ['HOSPITAL_NAME', 'CITY', 'STATE', 'COUNTRY']

# Mapping of known column variants
COLUMN_MAPPINGS = {
    'hospital name': 'HOSPITAL_NAME',
    'facility.name': 'HOSPITAL_NAME',
    'name': 'HOSPITAL_NAME',
    'city': 'CITY',
    'facility.city': 'CITY',
    'state': 'STATE',
    'facility.state': 'STATE',
    'state code': 'STATE',
    'country': 'COUNTRY',
}

In [24]:
# Helper to normalize column names
def normalize_columns(df):
    df.columns = [col.strip().lower() for col in df.columns]
    df.rename(columns={col: COLUMN_MAPPINGS[col] for col in df.columns if col in COLUMN_MAPPINGS}, inplace=True)
    return df

# Collect all rows into this list
all_data = []

In [31]:
# Step 1: Read all files from /data/*
for folder in glob.glob("./data/*"):
    for filepath in glob.glob(os.path.join(folder, "*")):
        try:
            ext = os.path.splitext(filepath)[1].lower()
            if ext == ".csv":
                df = pd.read_csv(filepath)
            elif ext in [".xls", ".xlsx"]:
                df = pd.read_excel(filepath)
            elif ext == ".json":
                with open(filepath, 'r') as f:
                    data = json.load(f)
                    if isinstance(data, dict) and 'hospitals' in data:
                        # Special case for file 4
                        df = pd.DataFrame(data['hospitals'], columns=['HOSPITAL_NAME'])
                    else:
                        df = pd.json_normalize(data)
            else:
                continue

            df = normalize_columns(df)

            # Keep only columns we care about
            subset_df = pd.DataFrame()
            for col in TARGET_COLUMNS:
                if col in df.columns:
                    subset_df[col] = df[col]
                else:
                    subset_df[col] = None

            all_data.append(subset_df)

        except Exception as e:
            print(f"Error reading {filepath}: {e}")

In [32]:
# Step 2: Combine, clean, and deduplicate
combined_df = pd.concat(all_data, ignore_index=True)

# Drop rows without hospital name
combined_df.dropna(subset=['HOSPITAL_NAME'], inplace=True)

# Remove duplicates based on hospital name
combined_df.drop_duplicates(subset=['HOSPITAL_NAME'], inplace=True)

In [34]:
# Fix this function to accept a single value
def infer_country(value):
    if pd.notna(value) and str(value).strip():
        return value
    return 'USA'

# Apply it to the column (correctly now)
combined_df['COUNTRY'] = combined_df['COUNTRY'].apply(infer_country)

# Final cleanup
final_df = combined_df[TARGET_COLUMNS].dropna(subset=['CITY', 'STATE'])

In [36]:
# Step 3: Write to output.csv

# Create directory if it doesn't exist
os.makedirs('output', exist_ok=True)

# Now save the file
output_path = './output/output.csv'
final_df.to_csv(output_path, index=False)

print(f"Final output saved to {output_path}")

Final output saved to ./output/output.csv


In [41]:
# Step 4: Load into MongoDB
def read_mongo_uri(config_file):
    with open(config_file, 'r') as f:
        config = json.load(f)
    return config['mongo_uri']

# Read URI and connect
mongo_uri = read_mongo_uri("./config/db_config.json")
client = MongoClient(mongo_uri)

# Access database and collection
db = client['loaded_hospitals']
collection = db['hospitals']

# Clean existing data
collection.delete_many({})

# Insert new data
collection.insert_many(final_df.to_dict(orient='records'))

print("✅ Data successfully loaded into MongoDB collection 'loaded_hospitals.hospitals'")

✅ Data successfully loaded into MongoDB collection 'loaded_hospitals.hospitals'
